In [599]:
%config Completer.use_jedi = False

In [600]:
import sys

In [601]:
sys.path.append("../scr")

In [602]:
import sys
from datetime import datetime, timedelta

import altair as alt
import numpy as np
import pandas as pd
import panel as pn
import yfinance as yf
from altair import datum
from utility import (
    plot_transactions,
    plot_transactions_2,
    read_ticker_ts,
    read_transactions,
)

from datetime import date, timedelta

In [603]:
import plotly.graph_objs as go
import plotly.express as px

In [604]:
fln = "../data/trasaction_history_18022021.csv"
tr = read_transactions(fln) 

# tr.Ticker.unique()

# ticker = "DIS"
# subset = tr[tr["Ticker"] == ticker]
# ts = read_ticker_ts(ticker=ticker, start="2020-5-1", end="2021-03-01")

## monthly transaction overview

Hereby I will create an overview of total transacations

In [605]:
mt = (
    tr.groupby(by=[pd.Grouper(key="Time", freq="M"), "action"])["action"]
    .count()
    .rename("transactions")
    .reset_index()
) # montly transaction

In [606]:
fig = go.Figure()

# reformat the date_time to provide a monthly summary (do not show the date)
mt['mnth_yr'] = mt['Time'].apply(lambda x: x.strftime('%b-%Y')) 

for action in ['buy', 'sell']:
    mt_ = mt.loc[mt['action']== action]
    fig.add_trace(go.Bar(
        x = mt_.mnth_yr,
        y = mt_.transactions,
        name = action,
    ))

fig.update_layout(barmode='group', xaxis_tickangle= -45, title_text='monthly transactions overview',
                 yaxis=dict(
        title='transaction counts',
        ))
fig.show()

## latest portofolio

1. what are the stocks in my pf?
1. total value (weekly overview)

## what are stocks in my pf, to today

as the first step, i would like to know what are the stocks in my current portofolio. i will use the complete df to summarize the transaction activities for sell and buy. the difference betwee these two columns is therefore the number of stocks that i am in possession. 

i will factorize it to a function that i can use to provide a summary on a rolling basis. 

In [607]:
tr_tickers = tr.groupby(by= ['Ticker', 'action'])['No. of shares'].sum().reset_index()

create a pivot table to convert the data from long to wide, afterwards clean the data and replace nan using zeros

In [608]:
pt = tr_tickers.pivot_table(values = 'No. of shares', index = 'Ticker', columns ='action').reset_index()

returned pivot table contains unwatned column, 'dividend' and nans. i will hereby drop and replace it

In [609]:
try:
    pt.drop("Dividend (Ordinary)", inplace = True, axis = 1)
except:
    pass

In [610]:
pt = pt.fillna(0)

In [611]:
pt['shares'] = pt['buy'] - pt['sell']
pt = pt.loc[pt['shares']>0.1]  # leave out ones already sold or only has a very small fraction

### read ticker history from yfiance

In [612]:
tickers = tr.Ticker.dropna().unique()
tickers = tickers.tolist()

start = tr.Time.min()
end = tr.Time.max() +  timedelta(1)

data = yf.download(tickers, start, end)

[*********************100%***********************]  42 of 42 completed

6 Failed downloads:
- LTM: No data found, symbol may be delisted
- INRG: No data found for this date range, symbol may be delisted
- AF: No data found for this date range, symbol may be delisted
- VUSA: No data found for this date range, symbol may be delisted
- IITU: No data found, symbol may be delisted
- RDSA: No data found, symbol may be delisted


remark: add a note for the stocks that were not presented due to data not available

### portofolio history

use the ticker time history and the transaction time history to create a portofolio time history

In [638]:
# determine number of shares for each day
def share_no_history(tr_pivoted, ticker):
    """determine the share no of each ticker through pivoting the transaction history
    """
    tr_sub = tr_pivoted.loc[ticker].dropna(subset = ['buy', 'sell'], how = 'all').reset_index()
    tr_sub = tr_sub.sort_values('Time')
    tr_sub.rename({'Time':'time_tr'},axis=1
                  ,inplace =True)
    tr_sub.drop(['Deposit','Dividend (Ordinary)','Withdrawal'], axis=1, inplace=True)
    return tr_sub

In [639]:
# retrieve stock value at close 
def ticker_price_history(data, ticker):
    """filter time history of the specifed ticker, history of ticker close price """
    tts_sub = data[[('Close',ticker)]]  # ticker time series
    tts_sub = tts_sub.reset_index()
    tts_sub.columns = tts_sub.columns.droplevel()
    tts_sub.columns = ['time_ts', 'close_price'] 
    
    return tts_sub

In [640]:
def merge_histories(tr_sub, tts_sub, ticker):
    """merge dataframe based on date and time
    
    note: invested amount is determined based on the market price at close, rather than the transaction record
    """
    merged = tts_sub.merge(right=tr_sub, left_on='time_ts', right_on='time_tr', how = "outer")
    merged[['buy', 'sell']]=merged[['buy', 'sell']].fillna(0) #, inplace=True, subset = ['buy', 'sell'], how = 'all')
    merged['no_shares'] = (merged['buy']-merged['sell']).cumsum()
    merged['invested']= (merged['close_price']*(merged['buy']-merged['sell'])).cumsum()
    merged['values'] = merged['close_price']*merged['no_shares'] 
    merged['ticker'] = ticker
    return merged

In [641]:
# test to merge and calculate the invested amount

In [642]:
tr_pivoted = tr.pivot_table(index=['Ticker', 'Time'], 
                            columns='action', values='No. of shares',dropna=False)

In [643]:
# loop it through for all the tickers: 
dfs = []

for ticker in tickers:
    
    tr_sub = share_no_history(tr_pivoted,ticker)
    tts_sub = ticker_price_history(data, ticker)
    df = merge_histories(tr_sub, tts_sub, ticker)
    
    dfs.append(df)
    
df_combined = pd.concat(dfs)

In [644]:
df_combined

,time_ts,close_price,time_tr,buy,sell,no_shares,invested,values,ticker
0,2020-04-30,108.150002,NaT,0.0,0.0,0.0,0.000000,0.000000,DIS
1,2020-05-01,105.500000,2020-05-01,9.0,0.0,9.0,949.500000,949.500000,DIS
2,2020-05-04,103.180000,NaT,0.0,0.0,9.0,949.500000,928.620003,DIS
3,2020-05-05,101.059998,NaT,0.0,0.0,9.0,949.500000,909.539978,DIS
4,2020-05-06,100.879997,NaT,0.0,0.0,9.0,949.500000,907.919975,DIS
...,...,...,...,...,...,...,...,...,...
198,2021-02-11,298.730011,NaT,0.0,0.0,0.0,0.000000,0.000000,SNOW
199,2021-02-12,299.470001,NaT,0.0,0.0,0.0,0.000000,0.000000,SNOW
200,2021-02-16,294.549988,NaT,0.0,0.0,0.0,0.000000,0.000000,SNOW
201,2021-02-17,288.779999,NaT,0.0,0.0,0.0,0.000000,0.000000,SNOW


### pie chart of portofolio composition

In [645]:
df_date = df_combined.loc[(df_combined['time_ts']== end- timedelta(1))&(df_combined['no_shares']>0.25)]

In [646]:
import plotly.express as px
# This dataframe has 244 lines, but 4 distinct values for `day`
# df = px.data.tips()
fig = px.pie(df_date, values='values', names='ticker')
fig.show()

### line plot for portofolio sums

In [647]:
df_agg = df_combined.pivot_table(index='time_ts', values=['values','invested'], aggfunc='sum').reset_index()

In [648]:
df = px.data.stocks()

fig = px.line(df_agg, x="time_ts", y= ['values','invested'],
              hover_data={"time_ts": "|%B %d, %Y"},
              title='Overview'
             )

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.show()

### deposit and withdraw information

In [691]:
tr_dw = tr[tr['Action'].isin(['Deposit', 'Withdrawal','Dividend (Ordinary)'])]
tr_dw = tr_dw.pivot_table(index=['Time'], 
                            columns='Action', values=['Total (EUR)'],dropna=False).reset_index()
tr_dw = tr_dw.droplevel(level=0, axis = 1)

In [669]:
investment_return = tr['Result (EUR)'].dropna().sum()

In [627]:
# example 

import plotly.express as px

df = px.data.stocks(indexed=True)-1
fig = px.area(df, facet_col="company", facet_col_wrap=2)
fig.show()